In [1]:
import os
import sys
import torch
from torch.utils.data import DataLoader
from torchvision.transforms.functional import normalize, to_pil_image
import matplotlib.pyplot as plt
import json
import math
import pandas as pd
import numpy as np

parent_dir = os.path.abspath(os.path.pardir)
sys.path.append(parent_dir)

import datasets
from models import get_model
from utils import resize_density_map, sliding_window_predict

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")


truncation = 4
reduction = 8
granularity = "fine"
anchor_points = "average"

model_name = "clip_vit_b_16"
input_size = 224
window_size = 16#224
stride = 16#224 #Usado con sliding window (PARECE FUNCIONAR MUCHO MEJOR EN LAS PREDICCIONES [mirar texto negro sobre las imagenes])
#stride=None 
weight_count_loss = 1.0
count_loss = "dmcount"

# Comment the lines below to test non-CLIP models.
prompt_type = "word"
num_vpt = 32
vpt_drop = 0.
deep_vpt = True

mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
alpha = 0.8

C:\Users\34628\Anaconda3\envs\CLIP\Lib\site-packages\torch\amp\autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


In [2]:
#CARGO LOS PUNTOS DE HINRICHS: 
#(Pruebo con Hinrichs primero que ya los conozco y se usarlos)
permutations = {2: (0, 1),
                3: (0, 1, 2),
                4: (0, 1, 3, 2),
                5: (0, 2, 4, 1, 3),
                6: (0, 2, 4, 1, 5, 3),
                7: (0, 2, 4, 6, 1, 3, 5),
                8: (0, 3, 6, 1, 4, 7, 2, 5),
                9: (0, 2, 6, 3, 8, 5, 1, 7, 4),
                10: (0, 3, 7, 1, 4, 9, 6, 2, 8, 5),
                11: (0, 3, 8, 1, 6, 10, 4, 7, 2, 9, 5),
                12: (0, 5, 10, 3, 8, 1, 6, 11, 4, 9, 2, 7),
                13: (0, 5, 10, 2, 7, 12, 4, 9, 1, 6, 11, 3, 8),
                14: (0, 5, 10, 2, 8, 13, 4, 11, 6, 1, 9, 3, 12, 7),
                15: (0, 4, 9, 13, 6, 1, 11, 3, 8, 14, 5, 10, 2, 12, 7),
                16: (0, 3, 11, 5, 14, 9, 1, 7, 12, 4, 15, 10, 2, 6, 13, 8),
                }

hinrichs = dict()
for i in range(2, 17):
    temp = []
    for j, k in zip(range(i), permutations[i]):
        temp.append((j/i, k/i))
    hinrichs[i] = temp

In [4]:
hinrichs

{2: [(0.0, 0.0), (0.5, 0.5)],
 3: [(0.0, 0.0),
  (0.3333333333333333, 0.3333333333333333),
  (0.6666666666666666, 0.6666666666666666)],
 4: [(0.0, 0.0), (0.25, 0.25), (0.5, 0.75), (0.75, 0.5)],
 5: [(0.0, 0.0), (0.2, 0.4), (0.4, 0.8), (0.6, 0.2), (0.8, 0.6)],
 6: [(0.0, 0.0),
  (0.16666666666666666, 0.3333333333333333),
  (0.3333333333333333, 0.6666666666666666),
  (0.5, 0.16666666666666666),
  (0.6666666666666666, 0.8333333333333334),
  (0.8333333333333334, 0.5)],
 7: [(0.0, 0.0),
  (0.14285714285714285, 0.2857142857142857),
  (0.2857142857142857, 0.5714285714285714),
  (0.42857142857142855, 0.8571428571428571),
  (0.5714285714285714, 0.14285714285714285),
  (0.7142857142857143, 0.42857142857142855),
  (0.8571428571428571, 0.7142857142857143)],
 8: [(0.0, 0.0),
  (0.125, 0.375),
  (0.25, 0.75),
  (0.375, 0.125),
  (0.5, 0.5),
  (0.625, 0.875),
  (0.75, 0.25),
  (0.875, 0.625)],
 9: [(0.0, 0.0),
  (0.1111111111111111, 0.2222222222222222),
  (0.2222222222222222, 0.6666666666666666),
  (

In [4]:
N ~ AreaTotal/AreaQuadrat * Media(PersonasContadasEnCadaPunto)

SyntaxError: invalid syntax (3798285359.py, line 1)

## Predictions on ShanghaiTech A (with GT labels)

In [3]:
##########################################prueba
for itera in range(224):
    window_size = itera
    stride = itera
    try:
        dataset_name = "sha" #select dataset name
        split = "val"
        
        if truncation is None:  # regression, no truncation.
            bins, anchor_points = None, None
        else:
            with open(os.path.join(parent_dir, "configs", f"reduction_{reduction}.json"), "r") as f:
                config = json.load(f)[str(truncation)][dataset_name]
            bins = config["bins"][granularity]
            anchor_points = config["anchor_points"][granularity]["average"] if anchor_points == "average" else config["anchor_points"][granularity]["middle"]
            bins = [(float(b[0]), float(b[1])) for b in bins]
            anchor_points = [float(p) for p in anchor_points]
        
        model = get_model(
            backbone=model_name,
            input_size=input_size, 
            reduction=reduction,
            bins=bins,
            anchor_points=anchor_points,
            # CLIP parameters
            prompt_type=prompt_type,
            num_vpt=num_vpt,
            vpt_drop=vpt_drop,
            deep_vpt=deep_vpt
        )
        
        #Change ckpt_dir_name to get the weights you want
        #ckpt_dir_name = f"{model_name}_{prompt_type}_" if "clip" in model_name else f"{model_name}_"
        #ckpt_dir_name += f"{input_size}_{reduction}_{truncation}_{granularity}_"
        #ckpt_dir_name += f"{weight_count_loss}_{count_loss}"
        ckpt_dir_name = 'ShanghaiTech_A_CLIP_EBC_ViT_B_16_Word'
        
        ckpt_path = os.path.join(
            parent_dir,
            "checkpoints",
            ckpt_dir_name,
            "best_mae.pth"  # select the weight file that you want to test
        )
        ckpt = torch.load(ckpt_path, map_location=device)
        model.load_state_dict(ckpt)
        model = model.to(device)

        dataset = datasets.Crowd(dataset=dataset_name, split=split, sigma=8, return_filename=True)
        dataloader = DataLoader(dataset, batch_size=1, shuffle=True, num_workers=0, collate_fn=datasets.collate_fn)
        data_iter = iter(dataloader)

        img_id = None
        model.eval() #No estoy seguro de si deberia meter esto en el bucle
        
        h16_sha_df_eval = pd.DataFrame() #Notación: h2 == hinrichs con n=2
        imagenes_recorridas = 0
        while True:
            try:
                image, points, density, image_path = next(data_iter)
                imagenes_recorridas += 1
            except: #Sólo debería entrar aquí si llega al final de las iteraciones
                print('Imagenes en el dataset: ', dataset.__len__())
                print('Imagenes recorridas: ', imagenes_recorridas)
                break
            
            original_image = image
            image_height, image_width = image.shape[-2:]
            #image = image.to(device)
            image_name = os.path.basename(image_path[0])
        
            #Recortar la imagen antes de evaluar el modelo (NO SE SI EL STRIDE IMPORTA O NO POR LA SLIDING WINDOW)
            N = 16 #NUMBER OF POINTS TO CONSIDER IN THE OPTIMAL POINT SET (tendre que hacer un bucle sobre range(N) para predecir modelo en cada recorte)
            pointset = hinrichs[N]
            cuentas = np.array([])
            
            for i in range(N): #Aqui deberia hacer bucle para cada punto...
                point = pointset[i]
                left = math.floor(point[0]*image_width)
                right = math.floor((point[0]+1/N)*image_width)
                bottom = math.floor(point[1]*image_height)
                top = math.floor((point[1]+1/N)*image_height)
                #Recorto la imagen tq la """"""""esquina inferior izquierda"""""""""" del recorte es el punto óptimo del pointset
                image = original_image[:,:,bottom:top,left:right] #OBS: Tamaño del recorte = máximo tq el recorte no se sobrepone a los demás ni se sale del tamaño
                new_image_height, new_image_width = image.shape[-2:]
                image = image.to(device)
            
                with torch.no_grad():
                    if stride is not None:  # Sliding window prediction.
                        pred_density = sliding_window_predict(model, image, window_size, stride)
                    else:
                        pred_density = model(image)
                    pred_count = pred_density.sum().item()
                    #resized_pred_density = resize_density_map(pred_density, (new_image_height, new_image_width)).cpu()
                print(itera)
                break
                #density = density.squeeze().numpy()
                #resized_pred_density = resized_pred_density.squeeze().numpy()
                #points = points[0].numpy() #EN SHA Y SHB POINTS PARECE SER YA UN ARRAY, EN QNRF DEBO DESCOMENTAR ESTA LINEA Y COMENTAR LA SIGUIENTE
                #points = points[0]
                cuentas = np.append(cuentas, pred_count)
            break
            
            h16_sha_df_eval.loc[image_name,'Pred_Count'] = N**2 * cuentas.mean() #((image_height*image_width) / ((image_height/N)*(image_width/N))) * cuentas.mean()
            h16_sha_df_eval.loc[image_name,'Pred_Count_Variance'] = np.var(N**2 * cuentas)
            #h16_sha_df_eval.loc[image_name,'GT Count'] = len(points)
    except:
        pass

################################################

Missing keys: []
Unexpected keys: ['proj']
All keys matched successfully.
Initialized model with text prompts: ['There is no person.', 'There is one person.', 'There are two people.', 'There are three people.', 'There are more than four people.']
Missing keys: []
Unexpected keys: ['proj']
All keys matched successfully.
Initialized model with text prompts: ['There is no person.', 'There is one person.', 'There are two people.', 'There are three people.', 'There are more than four people.']
Missing keys: []
Unexpected keys: ['proj']
All keys matched successfully.
Initialized model with text prompts: ['There is no person.', 'There is one person.', 'There are two people.', 'There are three people.', 'There are more than four people.']
Missing keys: []
Unexpected keys: ['proj']
All keys matched successfully.
Initialized model with text prompts: ['There is no person.', 'There is one person.', 'There are two people.', 'There are three people.', 'There are more than four people.']
Missing keys

In [3]:
dataset_name = "sha" #select dataset name
split = "val"

if truncation is None:  # regression, no truncation.
    bins, anchor_points = None, None
else:
    with open(os.path.join(parent_dir, "configs", f"reduction_{reduction}.json"), "r") as f:
        config = json.load(f)[str(truncation)][dataset_name]
    bins = config["bins"][granularity]
    anchor_points = config["anchor_points"][granularity]["average"] if anchor_points == "average" else config["anchor_points"][granularity]["middle"]
    bins = [(float(b[0]), float(b[1])) for b in bins]
    anchor_points = [float(p) for p in anchor_points]

model = get_model(
    backbone=model_name,
    input_size=input_size, 
    reduction=reduction,
    bins=bins,
    anchor_points=anchor_points,
    # CLIP parameters
    prompt_type=prompt_type,
    num_vpt=num_vpt,
    vpt_drop=vpt_drop,
    deep_vpt=deep_vpt
)

#Change ckpt_dir_name to get the weights you want
#ckpt_dir_name = f"{model_name}_{prompt_type}_" if "clip" in model_name else f"{model_name}_"
#ckpt_dir_name += f"{input_size}_{reduction}_{truncation}_{granularity}_"
#ckpt_dir_name += f"{weight_count_loss}_{count_loss}"
ckpt_dir_name = 'ShanghaiTech_A_CLIP_EBC_ViT_B_16_Word'

ckpt_path = os.path.join(
    parent_dir,
    "checkpoints",
    ckpt_dir_name,
    "best_mae.pth"  # select the weight file that you want to test
)
ckpt = torch.load(ckpt_path, map_location=device)
model.load_state_dict(ckpt)
model = model.to(device)

Missing keys: []
Unexpected keys: ['proj']
All keys matched successfully.
Initialized model with text prompts: ['There is no person.', 'There is one person.', 'There are two people.', 'There are three people.', 'There are more than four people.']


In [4]:
dataset = datasets.Crowd(dataset=dataset_name, split=split, sigma=8, return_filename=True)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True, num_workers=0, collate_fn=datasets.collate_fn)
data_iter = iter(dataloader)

In [18]:
import pandas as pd
#CODIGO PARA COMPROBAR QUE next(data_iter) RECORRE TODAS LAS IMAGENES BIEN
img_id = None #CODIGO PARA VER QUE EL RECORTE LE HAGO BIEN
model.eval()
aux=pd.Series()
for i in range(182): #Hay 182 imagenes en sha, si pongo range(183) da error de StopIteration
    image, points, density, image_path = next(data_iter) 
    aux[i] = (os.path.basename(image_path[0]))
len(aux.unique())

182

In [5]:
img_id = None
model.eval() #No estoy seguro de si deberia meter esto en el bucle

h16_sha_df_eval = pd.DataFrame() #Notación: h2 == hinrichs con n=2
imagenes_recorridas = 0
while True:
    try:
        image, points, density, image_path = next(data_iter)
        imagenes_recorridas += 1
    except: #Sólo debería entrar aquí si llega al final de las iteraciones
        print('Imagenes en el dataset: ', dataset.__len__())
        print('Imagenes recorridas: ', imagenes_recorridas)
        break
    
    original_image = image
    image_height, image_width = image.shape[-2:]
    #image = image.to(device)
    image_name = os.path.basename(image_path[0])

    #Recortar la imagen antes de evaluar el modelo (NO SE SI EL STRIDE IMPORTA O NO POR LA SLIDING WINDOW)
    N = 16 #NUMBER OF POINTS TO CONSIDER IN THE OPTIMAL POINT SET (tendre que hacer un bucle sobre range(N) para predecir modelo en cada recorte)
    pointset = hinrichs[N]
    cuentas = np.array([])
    
    for i in range(N): #Aqui deberia hacer bucle para cada punto...
        point = pointset[i]
        left = math.floor(point[0]*image_width)
        right = math.floor((point[0]+1/N)*image_width)
        bottom = math.floor(point[1]*image_height)
        top = math.floor((point[1]+1/N)*image_height)
        #Recorto la imagen tq la """"""""esquina inferior izquierda"""""""""" del recorte es el punto óptimo del pointset
        image = original_image[:,:,bottom:top,left:right] #OBS: Tamaño del recorte = máximo tq el recorte no se sobrepone a los demás ni se sale del tamaño
        new_image_height, new_image_width = image.shape[-2:]
        image = image.to(device)
    
        with torch.no_grad():
            if stride is not None:  # Sliding window prediction.
                pred_density = sliding_window_predict(model, image, window_size, stride)
            else:
                pred_density = model(image)
            pred_count = pred_density.sum().item()
            #resized_pred_density = resize_density_map(pred_density, (new_image_height, new_image_width)).cpu()
        
        #density = density.squeeze().numpy()
        #resized_pred_density = resized_pred_density.squeeze().numpy()
        #points = points[0].numpy() #EN SHA Y SHB POINTS PARECE SER YA UN ARRAY, EN QNRF DEBO DESCOMENTAR ESTA LINEA Y COMENTAR LA SIGUIENTE
        #points = points[0]
        cuentas = np.append(cuentas, pred_count)

    
    h16_sha_df_eval.loc[image_name,'Pred_Count'] = N**2 * cuentas.mean() #((image_height*image_width) / ((image_height/N)*(image_width/N))) * cuentas.mean()
    h16_sha_df_eval.loc[image_name,'Pred_Count_Variance'] = np.var(N**2 * cuentas)
    #h16_sha_df_eval.loc[image_name,'GT Count'] = len(points)

Imagenes en el dataset:  182
Imagenes recorridas:  182


In [ ]:
#El código de arriba tarda: 17 mins

In [ ]:
#Guardo los datos obtenidos para no tener que correr el bucle de nuevo si reinicio jupyter lab

In [6]:
%store h16_sha_df_eval

Stored 'h16_sha_df_eval' (DataFrame)


In [ ]:
#Cargo los datos obtenidos para no tener que correr el bucle de nuevo si reinicio jupyter lab

In [9]:
%store -r h16_sha_df_eval

In [7]:
h16_sha_df_eval

,Pred_Count,Pred_Count_Variance
002.jpg,5140.758420,1.681617e+07
056.jpg,1359.179073,1.312189e+06
025.jpg,500.798306,4.165729e+05
171.jpg,1954.810347,3.710589e+06
149.jpg,58.363895,4.486441e+03
...,...,...
123.jpg,1357.972394,1.206371e+06
069.jpg,1133.452609,5.672474e+05
045.jpg,6.965912,9.607255e+01
052.jpg,390.295158,1.900104e+05


## Predictions on ShanghaiTech B (with GT labels)

In [11]:
dataset_name = "shb" #select dataset name
split = "val"

if truncation is None:  # regression, no truncation.
    bins, anchor_points = None, None
else:
    with open(os.path.join(parent_dir, "configs", f"reduction_{reduction}.json"), "r") as f:
        config = json.load(f)[str(truncation)][dataset_name]
    bins = config["bins"][granularity]
    anchor_points = config["anchor_points"][granularity]["average"] if anchor_points == "average" else config["anchor_points"][granularity]["middle"]
    bins = [(float(b[0]), float(b[1])) for b in bins]
    anchor_points = [float(p) for p in anchor_points]

model = get_model(
    backbone=model_name,
    input_size=input_size, 
    reduction=reduction,
    bins=bins,
    anchor_points=anchor_points,
    # CLIP parameters
    prompt_type=prompt_type,
    num_vpt=num_vpt,
    vpt_drop=vpt_drop,
    deep_vpt=deep_vpt
)

#Change ckpt_dir_name to get the weights you want
#ckpt_dir_name = f"{model_name}_{prompt_type}_" if "clip" in model_name else f"{model_name}_"
#ckpt_dir_name += f"{input_size}_{reduction}_{truncation}_{granularity}_"
#ckpt_dir_name += f"{weight_count_loss}_{count_loss}"
ckpt_dir_name = 'ShanghaiTech_B_CLIP_EBC_ViT_B_16_Word'

ckpt_path = os.path.join(
    parent_dir,
    "checkpoints",
    ckpt_dir_name,
    "best_mae.pth"  # select the weight file that you want to test
)
ckpt = torch.load(ckpt_path, map_location=device)
model.load_state_dict(ckpt)
model = model.to(device)

Missing keys: []
Unexpected keys: ['proj']
All keys matched successfully.
Initialized model with text prompts: ['There is no person.', 'There is one person.', 'There are two people.', 'There are three people.', 'There are more than four people.']


In [12]:
dataset = datasets.Crowd(dataset=dataset_name, split=split, sigma=8, return_filename=True)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True, num_workers=0, collate_fn=datasets.collate_fn)
data_iter = iter(dataloader)

In [13]:
img_id = None
model.eval() #No estoy seguro de si deberia meter esto en el bucle

h16_shb_df_eval = pd.DataFrame()
imagenes_recorridas = 0
while True:
    try:
        image, points, density, image_path = next(data_iter)
        imagenes_recorridas += 1
    except: #Sólo debería entrar aquí si llega al final de las iteraciones
        print('Imagenes en el dataset: ', dataset.__len__())
        print('Imagenes recorridas: ', imagenes_recorridas)
        break
    
    original_image = image
    image_height, image_width = image.shape[-2:]
    #image = image.to(device)
    image_name = os.path.basename(image_path[0])

    #Recortar la imagen antes de evaluar el modelo (NO SE SI EL STRIDE IMPORTA O NO POR LA SLIDING WINDOW)
    N = 16 #NUMBER OF POINTS TO CONSIDER IN THE OPTIMAL POINT SET (tendre que hacer un bucle sobre range(N) para predecir modelo en cada recorte)
    pointset = hinrichs[N]
    cuentas = np.array([])
    
    for i in range(N): #Aqui deberia hacer bucle para cada punto...
        point = pointset[i]
        left = math.floor(point[0]*image_width)
        right = math.floor((point[0]+1/N)*image_width)
        bottom = math.floor(point[1]*image_height)
        top = math.floor((point[1]+1/N)*image_height)
        #Recorto la imagen tq la """"""""esquina inferior izquierda"""""""""" del recorte es el punto óptimo del pointset
        image = original_image[:,:,bottom:top,left:right] #OBS: Tamaño del recorte = máximo tq el recorte no se sobrepone a los demás ni se sale del tamaño
        new_image_height, new_image_width = image.shape[-2:]
        image = image.to(device)
    
        with torch.no_grad():
            if stride is not None:  # Sliding window prediction.
                pred_density = sliding_window_predict(model, image, window_size, stride)
            else:
                pred_density = model(image)
            pred_count = pred_density.sum().item()
            #resized_pred_density = resize_density_map(pred_density, (new_image_height, new_image_width)).cpu()
        
        #density = density.squeeze().numpy()
        #resized_pred_density = resized_pred_density.squeeze().numpy()
        #points = points[0].numpy() #EN SHA Y SHB POINTS PARECE SER YA UN ARRAY, EN QNRF DEBO DESCOMENTAR ESTA LINEA Y COMENTAR LA SIGUIENTE
        #points = points[0]
        cuentas = np.append(cuentas, pred_count)

    
    h16_shb_df_eval.loc[image_name,'Pred_Count'] = N**2 * cuentas.mean() #((image_height*image_width) / ((image_height/N)*(image_width/N))) * cuentas.mean()
    h16_shb_df_eval.loc[image_name,'Pred_Count_Variance'] = np.var(N**2 * cuentas)
    #h16_shb_df_eval.loc[image_name,'GT Count'] = len(points)

Imagenes en el dataset:  316
Imagenes recorridas:  316


In [ ]:
#El código de arriba tarda: 35 mins

In [14]:
%store h16_shb_df_eval

Stored 'h16_shb_df_eval' (DataFrame)


In [ ]:
%store -r h16_shb_df_eval


In [15]:
h16_shb_df_eval

,Pred_Count,Pred_Count_Variance
297.jpg,135.625973,33639.839615
248.jpg,305.819116,229917.632282
244.jpg,222.180472,115513.136885
199.jpg,363.434854,167811.195605
167.jpg,123.448751,41968.400802
...,...,...
074.jpg,280.636463,161242.651720
133.jpg,283.990090,243407.517244
252.jpg,282.735517,206403.344569
039.jpg,263.313352,122041.705115


## Predictions on UCF-QNRF (with GT labels)

In [16]:
dataset_name = "qnrf" #select dataset name
split = "val"

if truncation is None:  # regression, no truncation.
    bins, anchor_points = None, None
else:
    with open(os.path.join(parent_dir, "configs", f"reduction_{reduction}.json"), "r") as f:
        config = json.load(f)[str(truncation)][dataset_name]
    bins = config["bins"][granularity]
    anchor_points = config["anchor_points"][granularity]["average"] if anchor_points == "average" else config["anchor_points"][granularity]["middle"]
    bins = [(float(b[0]), float(b[1])) for b in bins]
    anchor_points = [float(p) for p in anchor_points]

model = get_model(
    backbone=model_name,
    input_size=input_size, 
    reduction=reduction,
    bins=bins,
    anchor_points=anchor_points,
    # CLIP parameters
    prompt_type=prompt_type,
    num_vpt=num_vpt,
    vpt_drop=vpt_drop,
    deep_vpt=deep_vpt
)

#Change ckpt_dir_name to get the weights you want
#ckpt_dir_name = f"{model_name}_{prompt_type}_" if "clip" in model_name else f"{model_name}_"
#ckpt_dir_name += f"{input_size}_{reduction}_{truncation}_{granularity}_"
#ckpt_dir_name += f"{weight_count_loss}_{count_loss}"
ckpt_dir_name = 'UCF_QNRF_CLIP_EBC_ViT_B_16_Word'

ckpt_path = os.path.join(
    parent_dir,
    "checkpoints",
    ckpt_dir_name,
    "best_mae.pth"  # select the weight file that you want to test
)
ckpt = torch.load(ckpt_path, map_location=device)
model.load_state_dict(ckpt)
model = model.to(device)

Missing keys: []
Unexpected keys: ['proj']
All keys matched successfully.
Initialized model with text prompts: ['There is no person.', 'There is one person.', 'There are two people.', 'There are three people.', 'There are more than four people.']


In [17]:
dataset = datasets.Crowd(dataset=dataset_name, split=split, sigma=8, return_filename=True)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True, num_workers=0, collate_fn=datasets.collate_fn)
data_iter = iter(dataloader)

In [18]:
img_id = None
model.eval() #No estoy seguro de si deberia meter esto en el bucle

h16_qnrf_df_eval = pd.DataFrame()
imagenes_recorridas = 0
while True:
    try:
        image, points, density, image_path = next(data_iter)
        imagenes_recorridas += 1
    except: #Sólo debería entrar aquí si llega al final de las iteraciones
        print('Imagenes en el dataset: ', dataset.__len__())
        print('Imagenes recorridas: ', imagenes_recorridas)
        break
    
    original_image = image
    image_height, image_width = image.shape[-2:]
    #image = image.to(device)
    image_name = os.path.basename(image_path[0])

    #Recortar la imagen antes de evaluar el modelo (NO SE SI EL STRIDE IMPORTA O NO POR LA SLIDING WINDOW)
    N = 16 #NUMBER OF POINTS TO CONSIDER IN THE OPTIMAL POINT SET (tendre que hacer un bucle sobre range(N) para predecir modelo en cada recorte)
    pointset = hinrichs[N]
    cuentas = np.array([])
    
    for i in range(N): #Aqui deberia hacer bucle para cada punto...
        point = pointset[i]
        left = math.floor(point[0]*image_width)
        right = math.floor((point[0]+1/N)*image_width)
        bottom = math.floor(point[1]*image_height)
        top = math.floor((point[1]+1/N)*image_height)
        #Recorto la imagen tq la """"""""esquina inferior izquierda"""""""""" del recorte es el punto óptimo del pointset
        image = original_image[:,:,bottom:top,left:right] #OBS: Tamaño del recorte = máximo tq el recorte no se sobrepone a los demás ni se sale del tamaño
        new_image_height, new_image_width = image.shape[-2:]
        image = image.to(device)
    
        with torch.no_grad():
            if stride is not None:  # Sliding window prediction.
                pred_density = sliding_window_predict(model, image, window_size, stride)
            else:
                pred_density = model(image)
            pred_count = pred_density.sum().item()
            #resized_pred_density = resize_density_map(pred_density, (new_image_height, new_image_width)).cpu()
        
        #density = density.squeeze().numpy()
        #resized_pred_density = resized_pred_density.squeeze().numpy()
        #points = points[0]
        cuentas = np.append(cuentas, pred_count)

    
    h16_qnrf_df_eval.loc[image_name,'Pred_Count'] = N**2 * cuentas.mean() #((image_height*image_width) / ((image_height/N)*(image_width/N))) * cuentas.mean()
    h16_qnrf_df_eval.loc[image_name,'Pred_Count_Variance'] = np.var(N**2 * cuentas)
    #h16_qnrf_df_eval.loc[image_name,'GT Count'] = len(points)

Imagenes en el dataset:  334
Imagenes recorridas:  334


In [ ]:
#El código de arriba tarda: 1h y 54 mins

In [19]:
%store h16_qnrf_df_eval

Stored 'h16_qnrf_df_eval' (DataFrame)


In [ ]:
%store -r h16_qnrf_df_eval


In [20]:
h16_qnrf_df_eval

,Pred_Count,Pred_Count_Variance
085.jpg,6848.763651,1.368980e+07
008.jpg,2701.773753,6.765069e+06
226.jpg,6908.120884,2.928430e+07
215.jpg,3101.664412,1.736371e+07
117.jpg,8241.821045,8.125220e+06
...,...,...
089.jpg,6838.600346,4.705956e+07
077.jpg,4552.097808,8.489470e+06
221.jpg,4059.151546,4.948262e+06
163.jpg,2184.002296,9.573856e+06


In [ ]:
########################################################################################################################

In [ ]:
#No ejecutar más código, tarda mucho y no merece la pena porque no tengo la cantidad de personas en cada imagen

## Predictions on NWPU-Test (without GT labels)

In [6]:
if truncation is None:  # regression, no truncation.
    bins, anchor_points = None, None
else:
    with open(os.path.join(parent_dir, "configs", f"reduction_{reduction}.json"), "r") as f:
        config = json.load(f)[str(truncation)]["nwpu"]
    bins = config["bins"][granularity]
    anchor_points = config["anchor_points"][granularity]["average"] if anchor_points == "average" else config["anchor_points"][granularity]["middle"]
    bins = [(float(b[0]), float(b[1])) for b in bins]
    anchor_points = [float(p) for p in anchor_points]


model = get_model(
    backbone=model_name,
    input_size=input_size, 
    reduction=reduction,
    bins=bins,
    anchor_points=anchor_points,
    # CLIP parameters
    prompt_type=prompt_type,
    num_vpt=num_vpt,
    vpt_drop=vpt_drop,
    deep_vpt=deep_vpt
)


#Change ckpt_dir_name to get the weights you want
#ckpt_dir_name = f"{model_name}_{prompt_type}_" if "clip" in model_name else f"{model_name}_"
#ckpt_dir_name += f"{input_size}_{reduction}_{truncation}_{granularity}_"
#ckpt_dir_name += f"{weight_count_loss}_{count_loss}"
ckpt_dir_name = 'NWPU_CLIP_ViT_B_16_Word'


ckpt_path = os.path.join(
    parent_dir,
    "checkpoints",
    ckpt_dir_name,
    "best_mae.pth"  # select the weight file that you want to test
)

#ckpt_dir_name = f"{model_name}_{prompt_type}_" if "clip" in model_name else f"{model_name}_"
#ckpt_dir_name += f"{input_size}_{reduction}_{truncation}_{granularity}_"
#ckpt_dir_name += f"{weight_count_loss}_{count_loss}"

#ckpt_path = os.path.join(
#    parent_dir,
#    "checkpoints",
#    "nwpu",
#    ckpt_dir_name,
#    "best_mae_0.pth"  # select the weight file that you want to test
#)
ckpt = torch.load(ckpt_path, map_location=device)
model.load_state_dict(ckpt)
model = model.to(device)
model.eval()

dataset = datasets.NWPUTest(transforms=None, return_filename=True)

Missing keys: []
Unexpected keys: ['proj']
All keys matched successfully.
Initialized model with text prompts: ['There is no person.', 'There is one person.', 'There are two people.', 'There are three people.', 'There are more than four people.']


In [ ]:
#ESTE CODIGO TARDA COMO UN DIA ENTERO, YA QUE SON 1500 IMAGENES. (No usar)

In [7]:
dataset.__len__()

1500

In [5]:
h16_nwpu_df_eval = pd.DataFrame()
imagenes_recorridas = 0
for k in range(dataset.__len__()):
    image, image_path = dataset[k]
    print(k)
    imagenes_recorridas += 1

    image = image.unsqueeze(0)
    
    original_image = image
    image_height, image_width = image.shape[-2:]
    #image = image.to(device)
    image_name = os.path.basename(image_path[0])

    #Recortar la imagen antes de evaluar el modelo (NO SE SI EL STRIDE IMPORTA O NO POR LA SLIDING WINDOW)
    N = 16 #NUMBER OF POINTS TO CONSIDER IN THE OPTIMAL POINT SET (tendre que hacer un bucle sobre range(N) para predecir modelo en cada recorte)
    pointset = hinrichs[N]
    cuentas = np.array([])
    
    for i in range(N): #Aqui deberia hacer bucle para cada punto...
        point = pointset[i]
        left = math.floor(point[0]*image_width)
        right = math.floor((point[0]+1/N)*image_width)
        bottom = math.floor(point[1]*image_height)
        top = math.floor((point[1]+1/N)*image_height)
        #Recorto la imagen tq la """"""""esquina inferior izquierda"""""""""" del recorte es el punto óptimo del pointset
        image = original_image[:,:,bottom:top,left:right] #OBS: Tamaño del recorte = máximo tq el recorte no se sobrepone a los demás ni se sale del tamaño
        new_image_height, new_image_width = image.shape[-2:]
        image = image.to(device)
    
        with torch.no_grad():
            if stride is not None:  # Sliding window prediction.
                pred_density = sliding_window_predict(model, image, window_size, stride)
            else:
                pred_density = model(image)
            pred_count = pred_density.sum().item()
            #resized_pred_density = resize_density_map(pred_density, (new_image_height, new_image_width)).cpu()
        
        #density = density.squeeze().numpy()
        #resized_pred_density = resized_pred_density.squeeze().numpy()
        #points = points[0].numpy() #EN SHA Y SHB POINTS PARECE SER YA UN ARRAY, EN QNRF DEBO DESCOMENTAR ESTA LINEA Y COMENTAR LA SIGUIENTE
        #points = points[0]
        cuentas = np.append(cuentas, pred_count)

    
    h16_nwpu_df_eval.loc[image_name,'Pred_Count'] = N**2 * cuentas.mean() #((image_height*image_width) / ((image_height/N)*(image_width/N))) * cuentas.mean()
    h16_nwpu_df_eval.loc[image_name,'Pred_Count_Variance'] = np.var(N**2 * cuentas)
    #nwpu_df_eval.loc[image_name,'GT Count'] = len(points) #EN ESTE CASO NO HAY POINTS, COMO CALCULAN ELLOS EL MAE?????????????

print('Imagenes en el dataset: ', dataset.__len__())
print('Imagenes recorridas: ', imagenes_recorridas)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

KeyboardInterrupt: 

In [ ]:
%store h16_nwpu_df_eval 

In [ ]:
%store -r h16_nwpu_df_eval 
